In [23]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [24]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data_wo_stops.csv",index_col=0)

In [25]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price,travel_route
0,SpiceJet,SG-8709,Delhi,Evening,zero,Night,Mumbai,Economy,2.17,1,5953,Delhi-Mumbai
1,SpiceJet,SG-8157,Delhi,Early_Morning,zero,Morning,Mumbai,Economy,2.33,1,5953,Delhi-Mumbai
2,AirAsia,I5-764,Delhi,Early_Morning,zero,Early_Morning,Mumbai,Economy,2.17,1,5956,Delhi-Mumbai
3,Vistara,UK-995,Delhi,Morning,zero,Afternoon,Mumbai,Economy,2.25,1,5955,Delhi-Mumbai
4,Vistara,UK-963,Delhi,Morning,zero,Morning,Mumbai,Economy,2.33,1,5955,Delhi-Mumbai


#  first spilt data test and train after done oversampling only on train data and train model and test on original data....

In [34]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42,stratify=df['class'])

# Print the shapes of the train and test sets
print("Training set shape:", train_data.shape)
print("Testing set shape:", test_data.shape)

Training set shape: (240115, 12)
Testing set shape: (60029, 12)


In [35]:
from imblearn.over_sampling import RandomOverSampler

# Assuming you have a pandas DataFrame called 'data' with features and class column
# Separate the features from the class column
X = train_data.drop('class', axis=1)
y = train_data['class']

# Apply RandomOverSampler to the entire dataset
oversampler = RandomOverSampler()
X_resampled, y_resampled = oversampler.fit_resample(X, y)

# Create a new DataFrame with the resampled data
resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
resampled_data['class'] = y_resampled

# Print the resampled dataset
print(resampled_data)
resampled_data

          airline  flight source_city departure_time stops   arrival_time  \
0         Vistara  UK-963       Delhi        Morning   one  Early_Morning   
1          Indigo  6E-605     Chennai      Afternoon   one          Night   
2         Vistara  UK-708     Kolkata          Night   one        Morning   
3       Air_India  AI-607      Mumbai        Evening   one        Evening   
4        SpiceJet  SG-768   Bangalore      Afternoon   one          Night   
...           ...     ...         ...            ...   ...            ...   
330649  Air_India  AI-672     Chennai      Afternoon   one  Early_Morning   
330650    Vistara  UK-832     Chennai  Early_Morning   one          Night   
330651    Vistara  UK-836     Chennai        Morning   one          Night   
330652    Vistara  UK-945       Delhi        Morning   one          Night   
330653  Air_India  AI-773     Kolkata        Evening   one        Morning   

       destination_city  duration  days_left  price       travel_route  \
0

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,duration,days_left,price,travel_route,class
0,Vistara,UK-963,Delhi,Morning,one,Early_Morning,Hyderabad,23.00,31,53937,Delhi-Hyderabad,Business
1,Indigo,6E-605,Chennai,Afternoon,one,Night,Hyderabad,7.33,21,3491,Chennai-Hyderabad,Economy
2,Vistara,UK-708,Kolkata,Night,one,Morning,Bangalore,15.42,11,11675,Kolkata-Bangalore,Economy
3,Air_India,AI-607,Mumbai,Evening,one,Evening,Hyderabad,26.25,8,8988,Mumbai-Hyderabad,Economy
4,SpiceJet,SG-768,Bangalore,Afternoon,one,Night,Delhi,5.50,23,5654,Bangalore-Delhi,Economy
...,...,...,...,...,...,...,...,...,...,...,...,...
330649,Air_India,AI-672,Chennai,Afternoon,one,Early_Morning,Hyderabad,16.33,25,40593,Chennai-Hyderabad,Business
330650,Vistara,UK-832,Chennai,Early_Morning,one,Night,Bangalore,13.50,7,60260,Chennai-Bangalore,Business
330651,Vistara,UK-836,Chennai,Morning,one,Night,Bangalore,12.58,34,60260,Chennai-Bangalore,Business
330652,Vistara,UK-945,Delhi,Morning,one,Night,Chennai,11.42,44,52392,Delhi-Chennai,Business


In [36]:
#removing colums which i dont want to use in model 
df1 = resampled_data.drop(['flight', 'travel_route'],axis=1)

#test data
df2 = test_data.drop(['flight', 'travel_route'],axis=1)

# get dummy variables for multiple columns
dummy_cols = ['airline', 'departure_time','arrival_time','source_city','destination_city','stops','class']
df1_dummy = pd.get_dummies(df1[dummy_cols])

#for test data, get dummy variables for multiple columns
df2_dummy = pd.get_dummies(df2[dummy_cols])

# concatenate original DataFrame with dummy variable DataFrame
df1_concat = pd.concat([df1.drop(dummy_cols, axis=1), df1_dummy], axis=1)
df2_concat = pd.concat([df2.drop(dummy_cols, axis=1), df2_dummy], axis=1)

# display result
df1_concat.head()
df2_concat.head()

#create x and y where x are features for the model and y as target
x_train = df1_concat.drop('price',axis=1)
y_train = df1_concat['price']

#for test data, create x and y where x are features for the model and y as target
x_test = df2_concat.drop('price',axis=1)
y_test = df2_concat['price']

In [37]:
def regression(x1,y1):
    import time

    start_time = time.time()

    #print shape of x train,y train, x test and y test 
    print('x_train shape:-',x_train.shape)
    print('x_test shape:-',x_test.shape)
    print('y_train shape:-',y_train.shape)
    print('y_test shape:-',y_test.shape)


    #import model from sklearn
    from sklearn.ensemble import RandomForestRegressor
    print('model name :- random forest regression')

    regr = RandomForestRegressor(random_state=123)

    #training model input as x_train and y_tarin
    regr.fit(x_train,y_train)

    print('score on the testing data :-',regr.score(x_test,y_test))
    print('score on the training data :-',regr.score(x_train,y_train))

    #finding error methods 
    from sklearn import metrics

    #model is creted show give x test data as input and retuen predictd data 
    predictions = regr.predict(x_test)

    #MAE
    print('MAE:-',metrics.mean_absolute_error(y_test,predictions))

    #MSE
    print('MSE:-',metrics.mean_squared_error(y_test,predictions))

    #RMSE
    print('RMSE:-',np.sqrt(metrics.mean_squared_error(y_test,predictions)))

    #r2
    print("r2",r2_score(y_test, predictions))

    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")
    
    return y_test,predictions

In [38]:
y_test,predictions = regression(x1,y1)

x_train shape:- (330654, 37)
x_test shape:- (60029, 37)
y_train shape:- (330654,)
y_test shape:- (60029,)
model name :- random forest regression
score on the testing data :- 0.9834858102977516
score on the training data :- 0.9986268522598257
MAE:- 1076.8262315595819
MSE:- 8545195.057093255
RMSE:- 2923.216560074408
r2 0.9834858102977516
Execution time: 80.97115063667297 seconds
